### Adicionando as bibliotecas

In [ ]:
# ---- Elaborado por: Scarlet Barbosa e Maria Rita---- #
# ---- última revisão em: 15-08-23 ---- #

# importa bibliotecas

!pip install reportlab==3.6.11
!pip install tabulate
!pip install fpdf

### Criando DF dados BIC e Unid. Aut.

In [2]:
# importa biblioteca 'pandas'
import pandas as pd
import numpy as np

from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
import pandas as pd
from google.colab import files
import datetime

# armazena dados das planilhas BIC e Unidades Autonomas na varável
unidades = pd.read_csv("/content/Unidades Autônomas.csv", sep = ";")
bic = pd.read_csv("/content/BIC.csv", sep = ";")
edificacoes = pd.read_csv("/content/Edificações.csv", sep = ";")

unidades_clone = unidades[:]
bic_clone = bic[:]
edificacoes_clone = edificacoes[:]

edificacoes_final = edificacoes[['GlobalID','ParentGlobalID','Número da edificação','Status da edificação','Tipologia','Elevador','Posição','Conservação','Estrutura','Acabamento geral','Esquadria','Vagas cobertas para veículos','Piscina','Sauna','Vestiário','Churrasqueira/Bar coberto','Quadra de esporte','Escada rolante','Circuito interno de TV','Refrigeração Central','Número de pavimentos','Área da edificação','Comando da edificação','Observação']]
bic_final = bic_clone[['GlobalID','Inscrição do Lote','Data do cadastro','Ocupação','Tipo de Utilização','Utilização Predominante','Utilização Secundária','Situação','Condomínio','Nivelamento','Topografia','Adequação para ocupação','Limitação','Passeio','Número total de unidades no lote','Status','Cadastrador','Exclusão de marquise','Exclusão de telheiro','Edição lote','Edição polígono (edif)','Entrega']]
unidades_final = unidades_clone[['GlobalID','Número da unidade','Comando','Inscrição Anterior','Inscrição Atual','Patrimônio','Status do Cadastro','Nome Logradouro','Nº','Bairro','Município','UF','Nome do contribuinte','Número total de edificações na unidade','Tipo de contribuinte']]

unidades_final = unidades_final.rename(columns = {'Inscrição Anterior':'Inscrição do Lote'})

# Excluir os últimos 4 caracteres em cada valor da coluna
unidades_final['Inscrição do Lote'] = unidades_final['Inscrição do Lote'].str[:-4]

# Realizar a união dos DataFrames com base nas colunas de chave diferentes
resultado = pd.merge(unidades_final, edificacoes_final, left_on='GlobalID', right_on='ParentGlobalID', how='outer')
resultado_geral = pd.merge(bic_final, resultado, left_on='Inscrição do Lote', right_on='Inscrição do Lote', how='outer')

# Exportar o DataFrame para um arquivo Excel
nome_arquivo_excel = 'dados-join-final.xlsx'  # Nome do arquivo Excel a ser gerado
resultado_geral.to_excel(nome_arquivo_excel, index=False)  # O argumento index=False evita a inclusão do índice no Excel


### ------------- Conferir nº de pavimentos e tipo de condominio ------------- ###

# Filtrar e identificar possíveis erros
erros = resultado_geral[(resultado_geral['Número de pavimentos'] <= 3) & (resultado_geral['Condomínio'] != 'Não se aplica')]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas = ['Inscrição do Lote','Número de pavimentos','Condomínio']
erros_selecionados = erros[colunas_selecionadas]

print("Possíveis erros (Número de Pavimentos X Condomínio):")
print(erros_selecionados)

# ------------- Conferir tipo de Ocupação X Nº de Lotes na Unidade: ------------- ###

# Filtrar e identificar possíveis erros
erros1 = resultado_geral[(resultado_geral['Ocupação'] == 1) & (resultado_geral['Número total de edificações na unidade'] != 0)]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas1 = ['Inscrição do Lote','Ocupação','Número total de edificações na unidade']
erros_selecionados1 = erros1[colunas_selecionadas1]

print("Possíveis erros (Ocupação X Nº de Lotes na Unidade):")
print(erros_selecionados1)

### ------------- Conferência de campos vazios: ------------- ###

# Colunas pré-selecionadas para análise de campos vazios
colunas_pre_selecionadas = ['Inscrição do Lote','Ocupação','Tipo de Utilização','Utilização Predominante','Utilização Secundária','Situação','Condomínio','Nivelamento','Topografia','Adequação para ocupação','Limitação','Passeio','Número total de unidades no lote','Status','Exclusão de marquise','Exclusão de telheiro','Edição lote','Edição polígono (edif)','Número da unidade','Comando','Patrimônio','Status do Cadastro','Nome Logradouro','Nº','Bairro','Município','UF','Nome do contribuinte','Número total de edificações na unidade','Tipo de contribuinte','Status da edificação','Tipologia','Elevador','Posição','Conservação','Estrutura','Acabamento geral','Esquadria','Vagas cobertas para veículos','Piscina','Sauna','Vestiário','Churrasqueira/Bar coberto','Quadra de esporte','Escada rolante','Circuito interno de TV','Refrigeração Central','Número de pavimentos','Comando da edificação']

# Inicializar uma lista para armazenar as informações de inscrições com campos vazios e colunas vazias
inscricoes_e_colunas_vazias = []

# Iterar pelas linhas do DataFrame
for index, row in resultado_geral.iterrows():
    if row['Comando da edificação'] != 'Cancelar':
        campos_vazios = [coluna for coluna in colunas_pre_selecionadas if pd.isna(row[coluna]) or row[coluna] == '']
        if campos_vazios:
            inscricoes_e_colunas_vazias.append((row['Inscrição do Lote'], campos_vazios))

print("Inscrições dos lotes com campos vazios e suas colunas vazias:")
for inscricao, colunas_vazias in inscricoes_e_colunas_vazias:
    print(f"Inscrição: {inscricao}, Colunas Vazias: {colunas_vazias}")

### ------------- Outras Conferências ------------- ###

### ------------- Ocorrências Insc. Lote x Nº Edificações: ------------- ###

# Filtre o DataFrame resultado_geral para remover as linhas onde 'Ocupação' é igual a 1
resultado_filtrado = resultado_geral[resultado_geral['Ocupação'] != 1]

# Calcule a contagem de ocorrências de cada número de inscrição no DataFrame filtrado
contagem_inscricoes = resultado_filtrado['Inscrição do Lote'].value_counts()

# Crie um DataFrame com os números de inscrição e a contagem de ocorrências
contagem_df = pd.DataFrame({'Inscrição do Lote': contagem_inscricoes.index, 'ocorrências': contagem_inscricoes.values})

# Crie uma função para obter a quantidade de edificações associada a uma inscrição
def obter_qtd_edificacoes(inscricao):
    return resultado_filtrado[resultado_filtrado['Inscrição do Lote'] == inscricao]['Número total de edificações na unidade'].iloc[0]

# Aplique a função para obter a quantidade de edificações para cada inscrição
contagem_df['Número total de edificações na unidade'] = contagem_df['Inscrição do Lote'].apply(obter_qtd_edificacoes)

# Filtrar os registros onde o número de ocorrências é diferente dos números de lotes
registros_diferentes = contagem_df[contagem_df['ocorrências'] != contagem_df['Número total de edificações na unidade']]

# Imprima a tabela com os registros onde o número de ocorrências é diferente dos números de lotes
print(registros_diferentes)

### ------------- Análise de Baldios x Comando da edificação: ------------- ###

erros2 = resultado_geral[(resultado_geral['Ocupação']==1) & (resultado_geral['Comando da edificação'] != 'Cancelar')]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas2 = ['Inscrição do Lote','Ocupação','Comando da edificação']
erros_selecionados2 = erros2[colunas_selecionadas2]

print("Possíveis erros (Ocupação X Comando da edificação):")
print(erros_selecionados2)

### ------------- Análise de Baldios x Comando da edificação: ------------- ###

erros3 = resultado_geral[(resultado_geral['Número da edificação'].isnull()) & (resultado_geral['Comando da edificação'] != 'Incluir')]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas3 = ['Inscrição do Lote','Número da edificação','Comando da edificação']
erros_selecionados3 = erros3[colunas_selecionadas3]

print("Possíveis erros (Número da edificação X Comando da edificação):")
print(erros_selecionados3)

### ------------- Análise Campos (Esquadria) ------------- ###

erros4 = resultado_geral[(resultado_geral['Esquadria'] == 'Sem/ Refugo/Reaproveitamento') & (resultado_geral['Ocupação'] != 4)]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas4 = ['Inscrição do Lote','Esquadria','Número da edificação']
erros_selecionados4 = erros4[colunas_selecionadas4]

print("Possíveis erros (Esquadria)")
print(erros_selecionados4)

### ------------- Análise (Ocupação x Nº Pavimentos) ------------- ###

erros5 = resultado_geral[(resultado_geral['Ocupação'] == 3) & (resultado_geral['Número de pavimentos'] == 0)]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas5 = ['Inscrição do Lote','Ocupação','Número de pavimentos']
erros_selecionados5 = erros5[colunas_selecionadas5]

print("Possíveis erros (Ocupação X Número de pavimentos):")
print(erros_selecionados5)

### ------------- BIC -  “Utilização Predominante” x “Utilização Secundária” ------------- ###

erros6 = resultado_geral[(resultado_geral['Utilização Predominante'] == 9) & (resultado_geral['Utilização Secundária'] != 9)]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas6 = ['Inscrição do Lote','Utilização Predominante','Utilização Secundária']
erros_selecionados6 = erros6[colunas_selecionadas6]

print("Possíveis erros (Utilização Predominante X Utilização Secundária):")
print(erros_selecionados6)

### ------------- BIC -  “Utilização Única x “Utilização Secundária” ------------- ###

erros7 = resultado_geral[(resultado_geral['Tipo de Utilização'] == 1) & (resultado_geral['Utilização Secundária'] != 9)]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas7 = ['Inscrição do Lote','Tipo de Utilização','Utilização Secundária']
erros_selecionados7 = erros7[colunas_selecionadas7]

print("Possíveis erros (Tipo de Utilização X Utilização Secundária):")
print(erros_selecionados7)

### ------------- BIC -  “Baldio x “Utilização Predominante” ------------- ###

erros8 = resultado_geral[(resultado_geral['Ocupação'] == 1) & (resultado_geral['Utilização Predominante'] != 9)]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas8 = ['Inscrição do Lote','Ocupação','Utilização Predominante']
erros_selecionados8 = erros8[colunas_selecionadas8]

print("Possíveis erros (Baldio X Utilização Predominante):")
print(erros_selecionados8)

### ------------- BIC -  Baldio x Tipo de Utilização ------------- ###

erros9 = resultado_geral[(resultado_geral['Ocupação'] == 1) & (resultado_geral['Tipo de Utilização'] != 1)]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas9 = ['Inscrição do Lote','Ocupação','Tipo de Utilização']
erros_selecionados9 = erros9[colunas_selecionadas9]

print("Possíveis erros (Baldio X Tipo de Utilização):")
print(erros_selecionados9)

### ------------- SALVANDO RESULTADOS EM UM ARQUIVO ------------- ###

from fpdf import FPDF
import pandas as pd
from google.colab import files

# Seu código para identificar possíveis erros e campos vazios

# Criar um arquivo PDF e adicionar os resultados
nome_cadastrador = resultado_geral['Entrega']
pdf_filename = "analise_resultados-NATHAN-24-08-23.pdf"
pdf = FPDF()
pdf.add_page()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.set_font("Arial", size=12)

# Função para dividir o texto e ajustá-lo para caber na célula da tabela
def adjust_text(text, max_width, font_size):
    words = text.split()
    adjusted_text = ""
    line = ""

    for word in words:
        test_line = line + word + " "
        test_width = pdf.get_string_width(test_line)

        if test_width <= max_width:
            line = test_line
        else:
            adjusted_text += line.strip() + "\n"
            line = word + " "

    adjusted_text += line.strip()
    return adjusted_text

# Função para adicionar resultados em uma tabela formatada
def add_table_to_pdf(title, data):
    pdf.set_font("Arial", "B", 14)
    pdf.cell(200, 10, title, ln=True, align='C')
    pdf.set_font("Arial", size=12)

    if not data.empty:
        col_widths = [40, 60, 80]  # Larguras das colunas ajustadas
        col_names = data.columns.tolist()
        row_height = 10

        # Adicionar cabeçalho da tabela
        pdf.set_fill_color(200, 220, 255)
        pdf.set_font("Arial", "B", 12)
        for col_name, col_width in zip(col_names, col_widths):
            pdf.cell(col_width, row_height, col_name, border=1, fill=True, align='C')
        pdf.ln()

        pdf.set_fill_color(255, 255, 255)  # Voltar à cor branca para as células de dados

        # Adicionar dados da tabela
        pdf.set_font("Arial", size=12)
        for row in data.values.tolist():
            row_values = []
            for idx, value in enumerate(row):
                col_width = col_widths[idx]
                if col_names[idx] == 'Número de pavimentos':
                    adjusted_value = adjust_text(str(value), col_width - 6, 12)  # Ajustar para caber na célula
                else:
                    adjusted_value = adjust_text(str(value), col_width - 6, 12)  # Ajustar para caber na célula
                row_values.append(adjusted_value)
            if not all(value.strip() == '' for value in row_values):
                for value, col_width in zip(row_values, col_widths):
                    pdf.cell(col_width, row_height, value, border=1, align='C')
                pdf.ln()
    else:
        pdf.cell(200, 10, "Nenhum erro encontrado nesta análise.", ln=True)

# Adicionar análise de Conferir nº de pavimentos e tipo de condominio
add_table_to_pdf("Conferir nº de pavimentos e tipo de condomínio", erros_selecionados)

# Adicionar análise de Conferir tipo de Ocupação X Nº de Lotes na Unidade
add_table_to_pdf("Conferir tipo de Ocupação e/ou Nº de Lotes na Unidade", erros_selecionados1)

# Adicionar análise de Análise de Baldios x Comando da edificação
add_table_to_pdf("Análise de Baldios x Comando da edificação", erros_selecionados2)

# Adicionar análise de Número total de edificações na unidade não preenchido e Comando diferente de Incluir
add_table_to_pdf("Número total de edificações não preenchido e Comando diferente de Incluir", erros_selecionados3)

# Conferir tipo de esquadrias:
add_table_to_pdf("Possíveis erros (Esquadria)", erros_selecionados4)

# Possíveis erros (Ocupação X Número de pavimentos):
add_table_to_pdf("Possíveis erros (Ocupação X Número de pavimentos):", erros_selecionados5)

# Possíveis erros (Utilização Predominante X Utilização Secundária):"
add_table_to_pdf("Possíveis erros (Utilização Predominante X Utilização Secundária):", erros_selecionados6)

# Possíveis erros (Utilização Predominante X Utilização Secundária):"
add_table_to_pdf("Possíveis erros (Tipo de Utilização X Utilização Secundária):", erros_selecionados7)

# Possíveis erros (Utilização Predominante X Utilização Secundária):"
add_table_to_pdf("Possíveis erros (Baldio X Utilização Predominante):", erros_selecionados8)

# Possíveis erros (Utilização Predominante X Utilização Secundária):"
add_table_to_pdf("Possíveis erros (Baldio X Tipo de Utilização):", erros_selecionados9)

# Adicionar análise de Ocorrências Insc. Lote x Nº Edificações
add_table_to_pdf("Ocorrências Insc. Lote x Nº Edificações", registros_diferentes)

# Adicionar análise de Conferência de campos vazios
pdf.set_font("Arial", "B", 14)
pdf.cell(200, 10, "Conferência de campos vazios", ln=True, align='C')
pdf.set_font("Arial", size=12)

if inscricoes_e_colunas_vazias:
    vazios_table_data = [['Inscrição do Lote', 'Campos sem Cadastro']] + inscricoes_e_colunas_vazias
    row_height = 10

    # Adicionar cabeçalho da tabela
    pdf.set_fill_color(200, 220, 255)
    pdf.set_font("Arial", "B", 12)
    for col_name, col_width in zip(vazios_table_data[0], [40, 150]):
        pdf.cell(col_width, row_height, col_name, border=1, fill=True, align='C')
    pdf.ln()

    pdf.set_fill_color(255, 255, 255)  # Voltar à cor branca para as células de dados

    # Adicionar dados da tabela
    pdf.set_font("Arial", size=12)
    for row in vazios_table_data[1:]:
        for value, col_width in zip(row, [40, 150]):
            adjusted_value = adjust_text(str(value), col_width - 6, 12)  # Ajustar para caber na célula
            pdf.cell(col_width, row_height, adjusted_value, border=1, align='C')
        pdf.ln()
else:
    pdf.cell(200, 10, "Nenhum campo vazio encontrado nesta análise.", ln=True)

# Salvar o arquivo PDF

pdf.output(pdf_filename)

# Baixar o arquivo PDF
files.download(pdf_filename)

Possíveis erros (Número de Pavimentos X Condomínio):
    Inscrição do Lote  Número de pavimentos  Condomínio
93     03.01.002.1123                   0.0    Vertical
102    03.01.015.0074                   1.0    Vertical
103    03.01.015.0074                   0.0    Vertical
104    03.01.015.0074                   0.0    Vertical
116    06.01.004.0161                   1.0    Vertical
121    06.01.010.0036                   2.0    Vertical
149    09.01.007.0143                   2.0  Horizontal
150    09.01.007.0143                   2.0  Horizontal
154    09.01.021.0050                   1.0  Horizontal
155    09.01.021.0050                   1.0  Horizontal
Possíveis erros (Ocupação X Nº de Lotes na Unidade):
    Inscrição do Lote  Ocupação  Número total de edificações na unidade
20     01.01.065.0012       1.0                                       1
54     03.01.001.0243       1.0                                       1
55     03.01.001.0244       1.0                               

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# PROGRAMA PARA CASOS: LIBERADO LEC

In [ ]:
# importa biblioteca 'pandas'
import pandas as pd
import numpy as np

from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
import pandas as pd
from google.colab import files
import datetime

# armazena dados das planilhas BIC e Unidades Autonomas na varável
unidades = pd.read_csv("/content/Unidades Autônomas.csv", sep = ";")
bic = pd.read_csv("/content/BIC.csv", sep = ";")
edificacoes = pd.read_csv("/content/Edificações.csv", sep = ";")

unidades_clone = unidades[:]
bic_clone = bic[:]
edificacoes_clone = edificacoes[:]

edificacoes_final = edificacoes[['GlobalID','ParentGlobalID','Número da edificação','Status da edificação','Tipologia','Elevador','Posição','Conservação','Estrutura','Acabamento geral','Esquadria','Vagas cobertas para veículos','Piscina','Sauna','Vestiário','Churrasqueira/Bar coberto','Quadra de esporte','Escada rolante','Circuito interno de TV','Refrigeração Central','Número de pavimentos','Área da edificação','Comando da edificação','Observação']]
bic_final = bic_clone[['GlobalID','Inscrição do Lote','Data do cadastro','Ocupação','Tipo de Utilização','Utilização Predominante','Utilização Secundária','Situação','Condomínio','Nivelamento','Topografia','Adequação para ocupação','Limitação','Passeio','Número total de unidades no lote','Status','Cadastrador','Exclusão de marquise','Exclusão de telheiro','Edição lote','Edição polígono (edif)','Entrega']]
unidades_final = unidades_clone[['GlobalID','Número da unidade','Comando','Inscrição Anterior','Inscrição Atual','Patrimônio','Status do Cadastro','Nome Logradouro','Nº','Bairro','Município','UF','Nome do contribuinte','Número total de edificações na unidade','Tipo de contribuinte']]

unidades_final = unidades_final.rename(columns = {'Inscrição Anterior':'Inscrição do Lote'})

# Excluir os últimos 4 caracteres em cada valor da coluna
unidades_final['Inscrição do Lote'] = unidades_final['Inscrição do Lote'].str[:-4]

# Realizar a união dos DataFrames com base nas colunas de chave diferentes
resultado = pd.merge(unidades_final, edificacoes_final, left_on='GlobalID', right_on='ParentGlobalID', how='outer')
resultado_geral = pd.merge(bic_final, resultado, left_on='Inscrição do Lote', right_on='Inscrição do Lote', how='outer')

# Exportar o DataFrame para um arquivo Excel
nome_arquivo_excel = 'dados-join-final.xlsx'  # Nome do arquivo Excel a ser gerado
resultado_geral.to_excel(nome_arquivo_excel, index=False)  # O argumento index=False evita a inclusão do índice no Excel


### ------------- Conferir nº de pavimentos e tipo de condominio ------------- ###

# Filtrar e identificar possíveis erros
erros = resultado_geral[(resultado_geral['Número de pavimentos'] <= 3) & (resultado_geral['Condomínio'] != 'Não se aplica')]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas = ['Inscrição do Lote','Número de pavimentos','Condomínio']
erros_selecionados = erros[colunas_selecionadas]

print("Possíveis erros (Número de Pavimentos X Condomínio):")
print(erros_selecionados)

# ------------- Conferir tipo de Ocupação X Nº de Lotes na Unidade: ------------- ###

# Filtrar e identificar possíveis erros
erros1 = resultado_geral[(resultado_geral['Ocupação'] == 1) & (resultado_geral['Número total de edificações na unidade'] != 0)]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas1 = ['Inscrição do Lote','Ocupação','Número total de edificações na unidade']
erros_selecionados1 = erros1[colunas_selecionadas1]

print("Possíveis erros (Ocupação X Nº de Lotes na Unidade):")
print(erros_selecionados1)

### ------------- Conferência de campos vazios: ------------- ###

# Colunas pré-selecionadas para análise de campos vazios
colunas_pre_selecionadas = ['Inscrição do Lote','Ocupação','Tipo de Utilização','Utilização Predominante','Situação','Condomínio','Nivelamento','Topografia','Adequação para ocupação','Limitação','Passeio','Número total de unidades no lote','Status','Exclusão de marquise','Exclusão de telheiro','Edição lote','Edição polígono (edif)','Número da unidade','Comando','Patrimônio','Status do Cadastro','Nome Logradouro','Nº','Bairro','Município','UF','Nome do contribuinte','Número total de edificações na unidade','Tipo de contribuinte','Status da edificação','Tipologia','Elevador','Posição','Conservação','Estrutura','Acabamento geral','Esquadria','Vagas cobertas para veículos','Piscina','Sauna','Vestiário','Churrasqueira/Bar coberto','Quadra de esporte','Escada rolante','Circuito interno de TV','Refrigeração Central','Número de pavimentos','Comando da edificação']

# Inicializar uma lista para armazenar as informações de inscrições com campos vazios e colunas vazias
inscricoes_e_colunas_vazias = []

# Iterar pelas linhas do DataFrame
for index, row in resultado_geral.iterrows():
    if row['Comando da edificação'] != 'Cancelar':
        campos_vazios = [coluna for coluna in colunas_pre_selecionadas if pd.isna(row[coluna]) or row[coluna] == '']
        if campos_vazios:
            inscricoes_e_colunas_vazias.append((row['Inscrição do Lote'], campos_vazios))

print("Inscrições dos lotes com campos vazios e suas colunas vazias:")
for inscricao, colunas_vazias in inscricoes_e_colunas_vazias:
    print(f"Inscrição: {inscricao}, Colunas Vazias: {colunas_vazias}")

### ------------- Outras Conferências ------------- ###

### ------------- Ocorrências Insc. Lote x Nº Edificações: ------------- ###

# Filtre o DataFrame resultado_geral para remover as linhas onde 'Ocupação' é igual a 1
resultado_filtrado = resultado_geral[resultado_geral['Ocupação'] != 1]

# Calcule a contagem de ocorrências de cada número de inscrição no DataFrame filtrado
contagem_inscricoes = resultado_filtrado['Inscrição do Lote'].value_counts()

# Crie um DataFrame com os números de inscrição e a contagem de ocorrências
contagem_df = pd.DataFrame({'Inscrição do Lote': contagem_inscricoes.index, 'ocorrências': contagem_inscricoes.values})

# Crie uma função para obter a quantidade de edificações associada a uma inscrição
def obter_qtd_edificacoes(inscricao):
    return resultado_filtrado[resultado_filtrado['Inscrição do Lote'] == inscricao]['Número total de edificações na unidade'].iloc[0]

# Aplique a função para obter a quantidade de edificações para cada inscrição
contagem_df['Número total de edificações na unidade'] = contagem_df['Inscrição do Lote'].apply(obter_qtd_edificacoes)

# Filtrar os registros onde o número de ocorrências é diferente dos números de lotes
registros_diferentes = contagem_df[contagem_df['ocorrências'] != contagem_df['Número total de edificações na unidade']]

# Imprima a tabela com os registros onde o número de ocorrências é diferente dos números de lotes
print(registros_diferentes)

### ------------- Análise de Baldios x Comando da edificação: ------------- ###

erros2 = resultado_geral[(resultado_geral['Ocupação']==1) & (resultado_geral['Comando da edificação'] != 'Cancelar')]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas2 = ['Inscrição do Lote','Ocupação','Comando da edificação']
erros_selecionados2 = erros2[colunas_selecionadas2]

print("Possíveis erros (Ocupação X Comando da edificação):")
print(erros_selecionados2)

### ------------- Análise de Baldios x Comando da edificação: ------------- ###

erros3 = resultado_geral[(resultado_geral['Número da edificação'].isnull()) & (resultado_geral['Comando da edificação'] != 'Incluir')]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas3 = ['Inscrição do Lote','Número da edificação','Comando da edificação']
erros_selecionados3 = erros3[colunas_selecionadas3]

print("Possíveis erros (Número da edificação X Comando da edificação):")
print(erros_selecionados3)

### ------------- Análise Campos (Esquadria) ------------- ###

erros4 = resultado_geral[(resultado_geral['Esquadria'] == 'Sem/ Refugo/Reaproveitamento') & (resultado_geral['Ocupação'] != 4)]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas4 = ['Inscrição do Lote','Esquadria','Número da edificação']
erros_selecionados4 = erros4[colunas_selecionadas4]

print("Possíveis erros (Esquadria)")
print(erros_selecionados4)

### ------------- Análise (Ocupação x Nº Pavimentos) ------------- ###

erros5 = resultado_geral[(resultado_geral['Ocupação'] == 3) & (resultado_geral['Número de pavimentos'] == 0)]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas5 = ['Inscrição do Lote','Ocupação','Número de pavimentos']
erros_selecionados5 = erros5[colunas_selecionadas5]

print("Possíveis erros (Ocupação X Número de pavimentos):")
print(erros_selecionados5)

### ------------- BIC -  “Utilização Predominante” x “Utilização Secundária” ------------- ###

erros6 = resultado_geral[(resultado_geral['Utilização Predominante'] == 9) & (resultado_geral['Utilização Secundária'].notna())]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas6 = ['Inscrição do Lote','Utilização Predominante','Utilização Secundária']
erros_selecionados6 = erros6[colunas_selecionadas6]

print("Possíveis erros (Utilização Predominante X Utilização Secundária):")
print(erros_selecionados6)

### ------------- BIC -  “Utilização Única x “Utilização Secundária” ------------- ###

erros7 = resultado_geral[(resultado_geral['Tipo de Utilização'] == 1) & (resultado_geral['Utilização Secundária'].notna())]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas7 = ['Inscrição do Lote','Tipo de Utilização','Utilização Secundária']
erros_selecionados7 = erros7[colunas_selecionadas7]

print("Possíveis erros (Tipo de Utilização X Utilização Secundária):")
print(erros_selecionados7)

### ------------- BIC -  “Baldio x “Utilização Predominante” ------------- ###

erros8 = resultado_geral[(resultado_geral['Ocupação'] == 1) & (resultado_geral['Utilização Predominante'] != 9)]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas8 = ['Inscrição do Lote','Ocupação','Utilização Predominante']
erros_selecionados8 = erros8[colunas_selecionadas8]

print("Possíveis erros (Baldio X Utilização Predominante):")
print(erros_selecionados8)

### ------------- BIC -  Baldio x Tipo de Utilização ------------- ###

erros9 = resultado_geral[(resultado_geral['Ocupação'] == 1) & (resultado_geral['Tipo de Utilização'] != 1)]

# Selecionar apenas colunas de interesse do DataFrame de erros
colunas_selecionadas9 = ['Inscrição do Lote','Ocupação','Tipo de Utilização']
erros_selecionados9 = erros9[colunas_selecionadas9]

print("Possíveis erros (Baldio X Tipo de Utilização):")
print(erros_selecionados9)

### ------------- SALVANDO RESULTADOS EM UM ARQUIVO ------------- ###

from fpdf import FPDF
import pandas as pd
from google.colab import files

# Seu código para identificar possíveis erros e campos vazios

# Criar um arquivo PDF e adicionar os resultados
nome_cadastrador = resultado_geral['Entrega']
pdf_filename = "liberado_lec-TEREZA-18-08-23.pdf"
pdf = FPDF()
pdf.add_page()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.set_font("Arial", size=12)

# Função para dividir o texto e ajustá-lo para caber na célula da tabela
def adjust_text(text, max_width, font_size):
    words = text.split()
    adjusted_text = ""
    line = ""

    for word in words:
        test_line = line + word + " "
        test_width = pdf.get_string_width(test_line)

        if test_width <= max_width:
            line = test_line
        else:
            adjusted_text += line.strip() + "\n"
            line = word + " "

    adjusted_text += line.strip()
    return adjusted_text

# Função para adicionar resultados em uma tabela formatada
def add_table_to_pdf(title, data):
    pdf.set_font("Arial", "B", 14)
    pdf.cell(200, 10, title, ln=True, align='C')
    pdf.set_font("Arial", size=12)

    if not data.empty:
        col_widths = [40, 60, 80]  # Larguras das colunas ajustadas
        col_names = data.columns.tolist()
        row_height = 10

        # Adicionar cabeçalho da tabela
        pdf.set_fill_color(200, 220, 255)
        pdf.set_font("Arial", "B", 12)
        for col_name, col_width in zip(col_names, col_widths):
            pdf.cell(col_width, row_height, col_name, border=1, fill=True, align='C')
        pdf.ln()

        pdf.set_fill_color(255, 255, 255)  # Voltar à cor branca para as células de dados

        # Adicionar dados da tabela
        pdf.set_font("Arial", size=12)
        for row in data.values.tolist():
            row_values = []
            for idx, value in enumerate(row):
                col_width = col_widths[idx]
                if col_names[idx] == 'Número de pavimentos':
                    adjusted_value = adjust_text(str(value), col_width - 6, 12)  # Ajustar para caber na célula
                else:
                    adjusted_value = adjust_text(str(value), col_width - 6, 12)  # Ajustar para caber na célula
                row_values.append(adjusted_value)
            if not all(value.strip() == '' for value in row_values):
                for value, col_width in zip(row_values, col_widths):
                    pdf.cell(col_width, row_height, value, border=1, align='C')
                pdf.ln()
    else:
        pdf.cell(200, 10, "Nenhum erro encontrado nesta análise.", ln=True)

# Adicionar análise de Conferir nº de pavimentos e tipo de condominio
add_table_to_pdf("Conferir nº de pavimentos e tipo de condomínio", erros_selecionados)

# Adicionar análise de Conferir tipo de Ocupação X Nº de Lotes na Unidade
add_table_to_pdf("Conferir tipo de Ocupação e/ou Nº de Lotes na Unidade", erros_selecionados1)

# Adicionar análise de Análise de Baldios x Comando da edificação
add_table_to_pdf("Análise de Baldios x Comando da edificação", erros_selecionados2)

# Adicionar análise de Número total de edificações na unidade não preenchido e Comando diferente de Incluir
add_table_to_pdf("Número total de edificações não preenchido e Comando diferente de Incluir", erros_selecionados3)

# Conferir tipo de esquadrias:
add_table_to_pdf("Possíveis erros (Esquadria)", erros_selecionados4)

# Possíveis erros (Ocupação X Número de pavimentos):
add_table_to_pdf("Possíveis erros (Ocupação X Número de pavimentos):", erros_selecionados5)

# Possíveis erros (Utilização Predominante X Utilização Secundária):"
add_table_to_pdf("Possíveis erros (Utilização Predominante X Utilização Secundária):", erros_selecionados6)

# Possíveis erros (Utilização Predominante X Utilização Secundária):"
add_table_to_pdf("Possíveis erros (Tipo de Utilização X Utilização Secundária):", erros_selecionados7)

# Possíveis erros (Utilização Predominante X Utilização Secundária):"
add_table_to_pdf("Possíveis erros (Baldio X Utilização Predominante):", erros_selecionados8)

# Possíveis erros (Utilização Predominante X Utilização Secundária):"
add_table_to_pdf("Possíveis erros (Baldio X Tipo de Utilização):", erros_selecionados9)

# Adicionar análise de Ocorrências Insc. Lote x Nº Edificações
add_table_to_pdf("Ocorrências Insc. Lote x Nº Edificações", registros_diferentes)

# Adicionar análise de Conferência de campos vazios
pdf.set_font("Arial", "B", 14)
pdf.cell(200, 10, "Conferência de campos vazios", ln=True, align='C')
pdf.set_font("Arial", size=12)

if inscricoes_e_colunas_vazias:
    vazios_table_data = [['Inscrição do Lote', 'Campos sem Cadastro']] + inscricoes_e_colunas_vazias
    row_height = 10

    # Adicionar cabeçalho da tabela
    pdf.set_fill_color(200, 220, 255)
    pdf.set_font("Arial", "B", 12)
    for col_name, col_width in zip(vazios_table_data[0], [40, 150]):
        pdf.cell(col_width, row_height, col_name, border=1, fill=True, align='C')
    pdf.ln()

    pdf.set_fill_color(255, 255, 255)  # Voltar à cor branca para as células de dados

    # Adicionar dados da tabela
    pdf.set_font("Arial", size=12)
    for row in vazios_table_data[1:]:
        for value, col_width in zip(row, [40, 150]):
            adjusted_value = adjust_text(str(value), col_width - 6, 12)  # Ajustar para caber na célula
            pdf.cell(col_width, row_height, adjusted_value, border=1, align='C')
        pdf.ln()
else:
    pdf.cell(200, 10, "Nenhum campo vazio encontrado nesta análise.", ln=True)

# Salvar o arquivo PDF

pdf.output(pdf_filename)

# Baixar o arquivo PDF
files.download(pdf_filename)

Possíveis erros (Número de Pavimentos X Condomínio):
Empty DataFrame
Columns: [Inscrição do Lote, Número de pavimentos, Condomínio]
Index: []
Possíveis erros (Ocupação X Nº de Lotes na Unidade):
     Inscrição do Lote  Ocupação  Número total de edificações na unidade
165     02.02.017.0102         1                                     1.0
501     14.01.031.0023         1                                     1.0
531     14.01.054.0010         1                                     1.0
532     14.01.054.0020         1                                     1.0
533     14.01.054.0030         1                                     1.0
559     14.01.055.0010         1                                     1.0
560     14.01.055.0041         1                                     1.0
573     14.01.060.0012         1                                     1.0
574     14.01.060.0020         1                                     1.0
575     14.01.060.0028         1                                     1.0
57

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>